<a href="https://colab.research.google.com/github/2021-FIIT-Bc-projects/BP-Eliseeva-Najdiskutovanejsie_temy_v_dokumentoch/blob/main/Bakalarka_NIPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 31.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 59 kB/s 
     |████████████████████████████████| 11.3 MB 38.1 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=13787f14741af5de22b1cacd530dd7c1ef85271ea6e9deda9648cfd3c8d25034
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take int

In [1]:
import gensim
import nltk
import pandas as pd
import re
import collections
import itertools
import warnings
import pyLDAvis
import pyLDAvis.gensim_models

from gensim import corpora
from sklearn.datasets import fetch_20newsgroups
from wordcloud import WordCloud
from nltk.corpus import stopwords
from gensim.models import CoherenceModel
from pprint import pprint

import matplotlib.pyplot as plt
import spacy

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa


In [6]:
import pkg_resources
print(pkg_resources.get_distribution('gensim').version)
print(pkg_resources.get_distribution('nltk').version)
print(pkg_resources.get_distribution('pandas').version)
print(pkg_resources.get_distribution('spacy').version)
print(pkg_resources.get_distribution('pyLDAvis').version)
print(pkg_resources.get_distribution('wordcloud').version)
print(pkg_resources.get_distribution('matplotlib').version)


3.6.0
3.2.5
1.3.4
2.2.4
3.3.1
1.5.0
3.2.2


In [7]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['let', 'fig'])
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def ShowWordCloud(data):
    string = ','.join(str(text) for text in data)
    # Create a WordCloud object
    wordcloud = WordCloud(background_color="white", max_words=5000, max_font_size=60, regexp=r"\w[\w']+\w").generate(string)

    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()


def sent_to_words(sentences):
    for sentence in sentences:
        yield gensim.utils.simple_preprocess(str(sentence), deacc=True)


def remove_stopwords_spacy(texts):
  filtered_texts = []
  for text in texts:
    filtered_sentence = [] 
    for word in text:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word)
    filtered_texts.append(filtered_sentence)
  return filtered_texts

def remove_stopwords_nltk(texts):
  filtered_texts = []
  for doc in texts:
    filtered_sentence = []
    for word in gensim.utils.simple_preprocess(str(doc)):
      if word not in stop_words:
        filtered_sentence.append(word)
    filtered_texts.append(filtered_sentence)
  return filtered_texts

def twenty_newsgroup_to_list():
    newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
    data = {'texts': newsgroups_train.data, 'topic_names': newsgroups_train['target_names'], 'topics_assigned': newsgroups_train['target']}
    return data

def remove_twosymbols(texts):
  for text in texts:
    for word in text:
      if len(word) <= 2:
        text.remove(word)
  return texts

def prepare_data(data, threshold):
  data_words = list(sent_to_words(data))

  data_words_nostops = remove_stopwords_nltk(data_words)

  data_words_nostops_shortRemoved = remove_twosymbols(data_words_nostops)

  bigram = gensim.models.Phrases(data_words_nostops_shortRemoved, threshold=threshold)
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  data_bigrams = [bigram_mod[text] for text in data_words_nostops_shortRemoved]

  data_lemmatized = []
  for text in data_bigrams:
    doc = nlp(" ".join(text))
    data_lemmatized.append([token.lemma_ for token in doc if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV']])
  
  return data_lemmatized

def clean_data(papers):
  papers = papers.drop(columns=['id', 'event_type', 'pdf_name', 'year', 'abstract'], axis=1)
  papers['paper_text_processed'] = papers['paper_text'].str.replace('[ \t\n\r\f\v]', " ")
  papers['paper_text_processed'] = papers['paper_text_processed'].str.replace(r"\S*@\S*\s?", "")
  papers['paper_text_processed'] = papers['paper_text_processed'].str.lower()
  return papers

In [34]:
papers_all = pd.read_csv('drive/MyDrive/Colab Notebooks/papers.csv')
papers_all = clean_data(papers_all)
data_all_lemmatized = prepare_data(papers_all['paper_text_processed'].values, 100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: FutureWarning: The default value of regex will change from True to False in a future version.


,title,paper_text,paper_text_processed
0,Self-Organization of Associative Database and ...,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,767 self-organization of associative database...
1,A Mean Field Theory of Layer IV of Visual Cort...,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,683 a mean field theory of layer iv of visual...
2,Storing Covariance by the Associative Long-Ter...,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,394 storing covariance by the associative lon...
3,Bayesian Query Construction for Neural Network...,Bayesian Query Construction for Neural\nNetwor...,bayesian query construction for neural network...
4,"Neural Network Ensembles, Cross Validation, an...","Neural Network Ensembles, Cross\nValidation, a...","neural network ensembles, cross validation, an..."
...,...,...,...
6555,Single Transistor Learning Synapses,Single Transistor Learning Synapses\n\nPaul Ha...,single transistor learning synapses paul hasl...
6556,"Bias, Variance and the Combination of Least Sq...","Bias, Variance and the Combination of\nLeast S...","bias, variance and the combination of least sq..."
6557,A Real Time Clustering CMOS Neural Engine,A Real Time Clustering CMOS\nNeural Engine\nT....,a real time clustering cmos neural engine t. s...
6558,Learning direction in global motion: two class...,Learning direction in global motion: two\nclas...,learning direction in global motion: two class...


In [33]:
papers = papers_all[:5000]
data_lemmatized = prepare_data(papers['paper_text_processed'].values, 100)
papers

,title,paper_text,paper_text_processed
0,Self-Organization of Associative Database and ...,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,767 self-organization of associative database...
1,A Mean Field Theory of Layer IV of Visual Cort...,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,683 a mean field theory of layer iv of visual...
2,Storing Covariance by the Associative Long-Ter...,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,394 storing covariance by the associative lon...
3,Bayesian Query Construction for Neural Network...,Bayesian Query Construction for Neural\nNetwor...,bayesian query construction for neural network...
4,"Neural Network Ensembles, Cross Validation, an...","Neural Network Ensembles, Cross\nValidation, a...","neural network ensembles, cross validation, an..."
...,...,...,...
4995,Low-Rank Time-Frequency Synthesis,Low-Rank Time-Frequency Synthesis\n\nMatthieu ...,low-rank time-frequency synthesis matthieu ko...
4996,A State-Space Model for Decoding Auditory Atte...,A State-Space Model for Decoding Auditory\nAtt...,a state-space model for decoding auditory atte...
4997,Efficient Structured Matrix Rank Minimization,Efficient Structured Matrix Rank Minimization\...,efficient structured matrix rank minimization ...
4998,Efficient Minimax Signal Detection on Graphs,Ef?cient Minimax Signal Detection on Graphs\n\...,ef?cient minimax signal detection on graphs j...


In [10]:
papers_test = papers[5000:]
data_test_lemmatized = prepare_data(papers_test['paper_text_processed'].values, 100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: FutureWarning: The default value of regex will change from True to False in a future version.


In [13]:
id2word = corpora.Dictionary(data_all_lemmatized)

# print(len(id2word))
id2word.filter_extremes(no_above=0.5, no_below=5)
# print(len(id2word))

corpus_train = [id2word.doc2bow(text) for text in data_lemmatized]
corpus_test = [id2word.doc2bow(text) for text in data_test_lemmatized]
corpus_all = [id2word.doc2bow(text) for text in data_all_lemmatized]

In [14]:
def measure_model(model):
  #perplexity
  print(model.log_perplexity(corpus_test))
  #coherence
  coherence_model_lda = CoherenceModel(model=model, texts=data_all_lemmatized, dictionary=id2word, coherence='c_v')
  print(coherence_model_lda.get_coherence())

In [15]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_train,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10)
pprint(lda_model.print_topics())

[(0,
  '0.096*"layer" + 0.049*"train" + 0.047*"unit" + 0.034*"deep" + '
  '0.023*"architecture" + 0.021*"net" + 0.019*"hide" + 0.016*"activation" + '
  '0.010*"recognition" + 0.009*"pool"'),
 (1,
  '0.015*"item" + 0.013*"event" + 0.013*"domain" + 0.009*"choice" + '
  '0.008*"rule" + 0.008*"individual" + 0.007*"mechanism" + 0.007*"concept" + '
  '0.007*"context" + 0.007*"effect"'),
 (2,
  '0.032*"subject" + 0.027*"region" + 0.021*"map" + 0.019*"target" + '
  '0.017*"location" + 0.015*"brain" + 0.014*"human" + 0.014*"trial" + '
  '0.010*"interaction" + 0.009*"visual"'),
 (3,
  '0.067*"policy" + 0.058*"action" + 0.042*"regret" + 0.035*"reward" + '
  '0.022*"agent" + 0.017*"arm" + 0.015*"strategy" + 0.014*"game" + '
  '0.013*"decision" + 0.013*"bandit"'),
 (4,
  '0.025*"prior" + 0.024*"inference" + 0.022*"latent" + 0.018*"gaussian" + '
  '0.016*"posterior" + 0.014*"mixture" + 0.013*"likelihood" + '
  '0.012*"observation" + 0.012*"density" + 0.010*"variational"'),
 (5,
  '0.058*"neuron" + 0

In [16]:
measure_model(lda_model)

-7.737959630082009
0.5324207271933366


In [17]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_all, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.121728  0.009873       1        1  10.380732
16     0.165973  0.067004       2        1   9.776610
14     0.155199  0.024113       3        1   7.802239
8      0.135547 -0.083701       4        1   6.166304
19     0.144267 -0.084903       5        1   6.099852
9      0.030592  0.004421       6        1   6.093406
15     0.112104  0.158797       7        1   5.616336
12    -0.102448 -0.216511       8        1   5.213122
1     -0.024274 -0.039482       9        1   4.990036
5     -0.231381  0.041485      10        1   4.880628
3      0.106736  0.105739      11        1   4.254284
18    -0.044484  0.138442      12        1   4.192310
10    -0.099914 -0.021586      13        1   3.876904
0     -0.034836 -0.006688      14        1   3.545999
13    -0.133060 -0.007620      15        1   3.332348
2     -0.223078 -0.037380      16        1   3.213869
17     0.045284 -0.203721      17        1   3.118524
6      0.031024  0.114097      18        1   2.895222
11     0.009816 -0.165700      19        1   2.786361
7     -0.164795  0.203323      20        1   1.764912, topic_info=           Term          Freq         Total Category  logprob  loglift
208       image  41513.000000  41513.000000  Default  30.0000  30.0000
1395    cluster  25276.000000  25276.000000  Default  29.0000  29.0000
582       layer  21444.000000  21444.000000  Default  28.0000  28.0000
470        word  18665.000000  18665.000000  Default  27.0000  27.0000
2932     policy  18029.000000  18029.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
1619      power    659.972886   4007.312513  Topic20  -5.1251   2.2334
654      signal    680.060770  12989.525774  Topic20  -5.0951   1.0874
872   threshold    606.234669   5463.189306  Topic20  -5.2100   1.8385
397      source    564.984112   7146.769381  Topic20  -5.2805   1.4995
941        gain    537.726579   4683.181978  Topic20  -5.3300   1.8727

[1350 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
2          3  0.039857    accelerate
2          4  0.026571    accelerate
2          7  0.800933    accelerate
2         18  0.130958    accelerate
3954      13  0.977147  acceleration
...      ...       ...           ...
2958      13  0.053244         world
2958      16  0.015473         world
16883     14  0.992260           wup
9582       2  0.995492            xe
6713      13  0.997730            yk

[4167 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 17, 15, 9, 20, 10, 16, 13, 2, 6, 4, 19, 11, 1, 14, 3, 18, 7, 12, 8])

In [18]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [19]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

--2021-11-18 11:49:17--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  7.93MB/s    in 1.9s    

2021-11-18 11:49:19 (7.93 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
   creating: mallet-2.0.8/
   creating: mallet-2.0.8/bin/
  inflating: mallet-2.0.8/bin/classifier2info  
  inflating: mallet-2.0.8/bin/csv2classify  
  inflating: mallet-2.0.8/bin/csv2vectors  
  inflating: mallet-2.0.8/bin/mallet  
  inflating: mallet-2.0.8/bin/mallet.bat  
  inflating: mallet-2.0.8/bin/mallethon  
  inflating: mallet-2.0.8/bin/prepend-license.sh  
  inflating: mallet-2.0.8/bin/svmlight2vectors  
  inflating: mallet-2.0.8/bin/text2classi

In [20]:
os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '/content/mallet-2.0.8/bin/mallet' # you should NOT need to change this 

In [22]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus_train, num_topics=20, id2word=id2word)
pprint(ldamallet.show_topics(formatted=False))

[(0,
  [('policy', 0.0615881990573381),
   ('action', 0.05858925441842918),
   ('reward', 0.034463986399690553),
   ('agent', 0.026684876818826496),
   ('reinforcement', 0.015052027869173428),
   ('decision', 0.012893551791487391),
   ('belief', 0.01057271246901966),
   ('cost', 0.010367370718266343),
   ('goal', 0.010348269160056732),
   ('iteration', 0.009985339554074124)]),
 (5,
  [('online', 0.028012527776319702),
   ('strategy', 0.027901714112411713),
   ('regret', 0.02695688182014359),
   ('game', 0.019975620993940242),
   ('player', 0.015379770090808882),
   ('cost', 0.014050006123913005),
   ('round', 0.011851229740054474),
   ('arm', 0.01064977633136785),
   ('play', 0.009751602423903091),
   ('expert', 0.009372503047375757)]),
 (14,
  [('density', 0.029651236068348366),
   ('variance', 0.027090873080177894),
   ('noise', 0.025189506472928933),
   ('gaussian', 0.024141594195070133),
   ('estimation', 0.023273735573200813),
   ('regression', 0.02270476602027404),
   ('estimator

In [23]:
import numpy as np
def mallet_to_lda(mallet_model):
    model_gensim = gensim.models.ldamodel.LdaModel(
        id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
        alpha=mallet_model.alpha, eta=0, iterations=1000,
        gamma_threshold=0.001,
        dtype=np.float32
    )
    model_gensim.sync_state()
    model_gensim.state.sstats = mallet_model.wordtopics
    return model_gensim

In [24]:
pyLDAvis.enable_notebook()
converted_mallet = mallet_to_lda(ldamallet)
vis = pyLDAvis.gensim_models.prepare(converted_mallet, corpus_all, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
9      0.135892 -0.101211       1        1  6.299446
18     0.217282 -0.161533       2        1  5.770890
4      0.051641 -0.003613       3        1  5.666239
6     -0.027588  0.183652       4        1  5.511140
16     0.235752 -0.024990       5        1  5.428373
3     -0.150942 -0.010823       6        1  5.424659
13     0.023531  0.001949       7        1  5.353628
8     -0.192737 -0.021936       8        1  5.262424
2     -0.105800  0.089980       9        1  5.234006
11    -0.114552  0.025320      10        1  5.056531
5     -0.076619 -0.086502      11        1  5.027586
14    -0.061385 -0.124739      12        1  4.992956
7      0.084654 -0.127352      13        1  4.634338
19    -0.182415 -0.109084      14        1  4.614632
15     0.128909  0.276107      15        1  4.560484
17    -0.010817  0.105853      16        1  4.430337
12    -0.110376  0.065294      17        1  4.400804
0     -0.076608 -0.088441      18        1  4.180454
10     0.049639  0.085525      19        1  4.167663
1      0.182538  0.026546      20        1  3.983410, topic_info=             Term          Freq         Total Category  logprob  loglift
208         image  40537.000000  40537.000000  Default  30.0000  30.0000
1395      cluster  26522.000000  26522.000000  Default  29.0000  29.0000
2403       kernel  20314.000000  20314.000000  Default  28.0000  28.0000
582         layer  20580.000000  20580.000000  Default  27.0000  27.0000
470          word  20129.000000  20129.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
747         event   1245.329793   7395.319590  Topic20  -5.3042   1.4416
967   measurement   1174.839428   5342.878364  Topic20  -5.3625   1.7084
343          real   1186.587822   8738.033003  Topic20  -5.3525   1.2264
1092  correlation   1158.056007   9365.777373  Topic20  -5.3769   1.1327
1547       domain   1163.091033   9988.169582  Topic20  -5.3725   1.0727

[1841 rows x 6 columns], token_table=       Topic      Freq               Term
term                                     
10632     12  0.994833            abalone
6731       2  0.997287          abdominal
7582       2  0.997287  abeles_corticonic
6155       9  1.031918         abound_new
2720      20  1.000988             accent
...      ...       ...                ...
4583       3  1.001046             writer
1383       3  0.967079             xerion
3297       1  1.030512    yale_university
3197       3  1.002245             zipser
17848      6  1.004661                znk

[4578 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 19, 5, 7, 17, 4, 14, 9, 3, 12, 6, 15, 8, 20, 16, 18, 13, 1, 11, 2])

In [25]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_all_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Coherence Score:  0.5620366606691015


In [31]:
def compute_coherence_values(corpus, texts, limit=25, start=5, step=2):
    model_list = []
    model_results = {'num_topics': [],
                      'alpha': [],
                      'coherence': []
                      }
    alpha = list(np.arange(0.01, 1, 0.3))

    for num_topics in range(start, limit, step):
      for a in alpha:
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word, alpha=a)
        model_list.append(model)
        coherence_model = CoherenceModel(model=model, texts=texts, dictionary=id2word, coherence='c_v')
        coh = coherence_model.get_coherence()
        model_results['num_topics'].append(num_topics)
        model_results['alpha'].append(a)
        model_results['coherence'].append(coh)
    return model_list, model_results

In [29]:
model_list, model_results = compute_coherence_values(corpus_train, data_all_lemmatized)
model_results

KeyboardInterrupt: ignored

In [ ]:
max_value = max(model_results['coherence'])
max_index = model_results['coherence'].index(max_value)
print(max_value)

a1 = {'num_topics': [], 'coherence': []} #alpha = 0.01
a2 = {'num_topics': [], 'coherence': []} #alpha = 0.31
a3 = {'num_topics': [], 'coherence': []} #alpha = 0.61
a4 = {'num_topics': [], 'coherence': []} #alpha = 0.91

for i in range (0,40):
  if model_results['alpha'][i] == 0.01:
    a1['num_topics'].append(model_results['num_topics'][i])
    a1['coherence'].append(model_results['coherence'][i])
  elif model_results['alpha'][i] == 0.31:
    a2['num_topics'].append(model_results['num_topics'][i])
    a2['coherence'].append(model_results['coherence'][i])
  elif model_results['alpha'][i] == 0.61:
    a3['num_topics'].append(model_results['num_topics'][i])
    a3['coherence'].append(model_results['coherence'][i])
  else:
    a4['num_topics'].append(model_results['num_topics'][i])
    a4['coherence'].append(model_results['coherence'][i])

plt.plot(a1['num_topics'], a1['coherence'])
plt.show()
plt.plot(a2['num_topics'], a2['coherence'])
plt.show()
plt.plot(a3['num_topics'], a3['coherence'])
plt.show()
plt.plot(a4['num_topics'], a4['coherence'])
plt.show()

print(model_results['coherence'][17])
print(model_results['num_topics'][17])
print(model_results['alpha'][17])